In [2]:
import numpy as np
import pandas as pd
from os import sep as sep

In [3]:
exp_path = r"Y:\Lior&Einav\Experiments\experiment11_140720"

In [4]:
fdata = pd.read_csv(exp_path+sep+'forager_table_with_feeding_sizes.csv')
weights = pd.read_excel(exp_path+sep+'weights.xlsx')

In [5]:
fdata.head()

,ant_id,feeding_start,feeding_end,food_source,last_interaction_before_end,last_interaction_before_partner,first_interaction_after_start,first_interaction_after_partner,crop_before,crop_after,feeding_size_intensity
0,76,48,102,yellow,43,-1,116,392,169935.671875,394976.556374,225040.884499
1,148,92,172,yellow,43,-1,277,76,60206.528134,339347.484375,279140.956241
2,289,177,348,yellow,43,-1,361,72,6457.684912,602614.614376,596156.929464
3,392,203,250,yellow,158,76,294,95,127725.829687,454063.550000,326337.720313
4,289,415,439,yellow,383,72,476,36,359116.687500,553075.133774,193958.446274


In [6]:
weights

,food,before_g,after_g,color,type
0,1_2,7.42178,6.98640,yellow,treatment
1,1_2_control,7.45094,7.38830,yellow,control
2,3_1,7.50066,7.29342,red,treatment
3,3_1_control,7.42361,7.35877,red,control


In [38]:
def get_consumed_weights(weights_df):
    weights_df['difference'] = weights_df['before_g']-weights_df['after_g']
    control_rows = weights_df['type']=='control'
    treatment_rows = ~control_rows
    evap = np.mean(weights_df['difference'][control_rows])
    consumed_g = {}
    for c in ['yellow','red']:
        consumed_g[c] = weights_df.difference[treatment_rows & (weights_df['color']==c)].to_numpy()[0] - evap
    return consumed_g
    
    

In [40]:
consumed_g = get_consumed_weights(weights)
consumed_g

{'yellow': 0.3716400000000002, 'red': 0.14349999999999952}

In [43]:
def get_consumed_intensities(feeding_df):
    consumed_i = feeding_df['feeding_size_intensity'].groupby(fdata['food_source']).sum()
    return consumed_i

In [45]:
consumed_i = get_consumed_intensities(fdata)
consumed_i

food_source
red       1.026401e+07
yellow    2.178459e+07
Name: feeding_size_intensity, dtype: float64

In [73]:
def get_coversion_factors(consumed_i,consumed_g):
    conversion_factors = {}
    for c in ['red','yellow']:
        ul = consumed_g[c]*1000
        conversion_factors[c] = consumed_i[c]/ul
    return pd.DataFrame([conversion_factors])

In [74]:
conversion_factors = get_coversion_factors(consumed_i,consumed_g)
conversion_factors

,red,yellow
0,71526.233111,58617.440982


### write conversion factors

In [75]:
conversion_factors.to_csv(exp_path+sep+'conversion_factors_by_weigth_and_feeding_sum.csv',index=False)